# CNN
 - image data를 그대로사용
 - 각 channel 별로 filter를 통과하여 합산(feature map 추출)
 - 하나의 filter에는 하나의 feature map이 나온다.
 - feature map을 actvation function을 통과하여 activation map 추출
 - activation map을 pooling layer 통과시킨다.
 - loss를 확인후 filter update
 - output size : (N - F) / stride + 1

In [1]:
# filter : convolution layer 
# stride : filter의 이동거리 stride값이 커질경우 output의 크기가 작아짐
# padding :손실을 최소화 하기 위해 사용(ex zero padding)
# pooling : convolution 과정을 통해 나온 결과값(feature map)의 dimentionality를 축소 (max pooling, average pooling)
# flatten : dense layer를 통과할수 있게 하기위한 과정 

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy 

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainset, valset = torch.utils.data.random_split(trainset, [40000, 10000])
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

partition = {'train': trainset, 'val':valset, 'test':testset}

Files already downloaded and verified
Files already downloaded and verified


In [6]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
}
# M = pooling layer

In [15]:
class CNN(nn.Module):
    def __init__(self, model_code, in_channel, out_dim, act, use_bn):
        super(CNN, self).__init__()
        if act == 'relu':
            self.act = nn.ReLU()
        elif act == 'sigmoid':
            self.act = nn.Sigmoid()
        elif act == 'tanh':
            self.act = TanH()
        else:
            raise ValueError('Not a valid activation function code')
        
        self.layers = self.make_layers(model_code, in_channel, use_bn)
        self.classifer = nn.Sequential(nn.Linear(512,256), self.act, nn.Linear(256, out_dim))
            
    def make_layers(self, model_code, in_channel, use_bn):
        layers = []
        for x in cfg[model_code]:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size = 2, stride = 2)]
            else:
                layers += [nn.Conv2d(in_channels = in_channel,
                                     out_channels = x, 
                                     kernel_size = 3,
                                     stride = 1,
                                     padding = 1)]
                if use_bn:
                    layers += [nn.BatchNorm2d(x)]
                layers += [self.act]
                in_channel = x
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.layers(x)
        print(x.size())
        x = x.view(x.size(0), -1)
        print(x.size())
        x = self.classifer(x)
        return x

In [16]:
def dimenison_check():
    model = CNN(model_code='VGG11',in_channel= 3,out_dim = 10, act = 'relu', use_bn= True)
    x = torch.randn(2,3, 32, 32) # 임의의 input
    y = model(x)
    print(y.size())

In [17]:
dimenison_check()

torch.Size([2, 512, 1, 1])
torch.Size([2, 512])
torch.Size([2, 10])


In [19]:

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform)
trainset, valset = torch.utils.data.random_split(trainset, [40000, 10000])
testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
partition = {'train': trainset, 'val':valset, 'test':testset}

100.0%


Extracting ./data\cifar-100-python.tar.gz to ./data
Files already downloaded and verified
